In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier, XGBRFClassifier

from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, matthews_corrcoef
from sklearn.preprocessing import label_binarize
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from lstm import LSTMNet, LSTMNetParams, prepare_data as prepare_data_lstm

from util.encoding import encode


Load data

In [5]:
df = pd.read_csv("data/features.csv")
X = df.iloc[:, 2:-1]  # skip index and name

y = df["label"]  # 10 genres
y, code = encode(y) # encode labels to integers

Prepare data

In [6]:
#### NORMALIZE X ####
# Normalize so everything is on the same scale.

cols = X.columns
std_scaler = sklearn.preprocessing.StandardScaler()
np_scaled = std_scaler.fit_transform(X)

# new data frame with the new scaled data. 
X = pd.DataFrame(np_scaled, columns = cols)

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
def model_assess(model, X_train, X_test, y_train, y_test, title="Default"):
    """
    Fit given model and assess its performance regarding accuracy, F1 score, AUC score and
    matthews correlation coefficient
    """
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="weighted")

    # binarize labels for multi class roc score
    labels = list(set(y))
    y_test_bin = label_binarize(y_test, classes=labels)
    y_pred_bin = label_binarize(y_pred, classes=labels)
    roc = roc_auc_score(y_test_bin, y_pred_bin, average="weighted", multi_class="ovo")
    matt_cor = matthews_corrcoef(y_test, y_pred)

    print(
        f"{title}:\n  Acc: {round(acc, 2)}\n  F1: {round(f1, 2)}\n  AUC score: {round(roc, 2)}\n  Matth. corr. coeff.: {round(matt_cor, 2)}"
    )
    return acc, f1, roc, matt_cor


### Classifier experiments

In [12]:
# save experiment results
df_res = pd.DataFrame(columns=["classifier", "accuracy", "F1", "AUC", "MCC"])

Logistic Regression

In [48]:
log_reg = LogisticRegression(random_state=0, max_iter=500).fit(X_train, y_train)
log_reg_acc, log_reg_f1, log_reg_auc, log_reg_mcc = model_assess(
    log_reg, X_train, X_test, y_train, y_test, title="Log. Regression"
)
df_res.loc[len(df_res)] = ["Logistic regression", log_reg_acc, log_reg_f1, log_reg_auc, log_reg_mcc]


Log. Regression:
  Acc: 0.59
  F1: 0.6
  ROC score: 0.77
  Matth. corr. coeff.: 0.55


Gaussian Naive bayes

In [73]:
gnb = GaussianNB()
gnb_acc, gnb_f1, gnb_auc, gnb_mcc = model_assess(
    gnb, X_train, X_test, y_train, y_test, title="Gaussian naive bayes"
)
df_res.loc[len(df_res)] = ["Gaussian naive bayes", gnb_acc, gnb_f1, gnb_auc, gnb_mcc]


Gaussian naive bayes:
  Acc: 0.45
  F1: 0.41
  ROC score: 0.7
  Matth. corr. coeff.: 0.4


Support vector classifier

In [51]:
svc_clf = SVC(gamma="auto")
svc_acc, svc_f1, svc_auc, svc_mcc = model_assess(
    svc_clf, X_train, X_test, y_train, y_test, title="SVC"
)
df_res.loc[len(df_res)] = ["SVC", svc_acc, svc_f1, svc_auc, svc_mcc]



Gaussian naive bayes:
  Acc: 0.69
  F1: 0.69
  ROC score: 0.83
  Matth. corr. coeff.: 0.66


Random forests

In [52]:
forest_clf = RandomForestClassifier()
forest_acc, forest_f1, forest_auc, forest_mcc = model_assess(
    forest_clf, X_train, X_test, y_train, y_test, title="Random forest"
)
df_res.loc[len(df_res)] = ["Random forest", forest_acc, forest_f1, forest_auc, forest_mcc]

Random forest:
  Acc: 0.69
  F1: 0.69
  ROC score: 0.83
  Matth. corr. coeff.: 0.66


XGBoost

In [71]:
xgb = XGBClassifier(
    n_estimators=1000,
    booster="gbtree",
    learning_rate=0.04,
    eval_metric="mlogloss",
    random_state=42,
)
xgb_acc, xgb_f1, xgb_auc, xgb_mcc = model_assess(
    xgb, X_train, X_test, y_train, y_test, title="XGBoost"
)
df_res.loc[len(df_res)] = ["XGBoost", xgb_acc, xgb_f1, xgb_auc, xgb_mcc]


/home/ivo/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBoost:
  Acc: 0.7
  F1: 0.7
  ROC score: 0.83
  Matth. corr. coeff.: 0.67


In [74]:
xgbrf = XGBRFClassifier(
    n_estimators=1000,
    booster="gbtree",
    learning_rate=0.04,
    objective="multi:softmax",
    eval_metric="mlogloss",
    random_state=42,
)
xgbrf_acc, xgbrf_f1, xgbrf_auc, xgbrf_mcc = model_assess(
    xgbrf, X_train, X_test, y_train, y_test, title="XGBoost random forests"
)
df_res.loc[len(df_res)] = ["XGBoost random forests", xgbrf_acc, xgbrf_f1, xgbrf_auc, xgbrf_mcc]


/home/ivo/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBoost:
  Acc: 0.68
  F1: 0.67
  ROC score: 0.82
  Matth. corr. coeff.: 0.64


MLP Sklearn

In [57]:
mlp = MLPClassifier(
    random_state=42,
    max_iter=1000,
    activation="tanh",
    solver="adam",
    alpha=0.0001,
    learning_rate="adaptive",
    learning_rate_init=0.01,
)
mlp_acc, mlp_f1, mlp_auc, mlp_mcc = model_assess(
    mlp, X_train, X_test, y_train, y_test, title="MLP"
)
df_res.loc[len(df_res)] = ["MLP", mlp_acc, mlp_f1, mlp_auc, mlp_mcc]


MLP:
  Acc: 0.73
  F1: 0.73
  ROC score: 0.85
  Matth. corr. coeff.: 0.7


LSTM Pytorch

In [13]:
X_train_t, X_test_t, y_train_t, y_test_t = prepare_data_lstm(
    X_train, X_test, y_train, y_test
)
lstm_params = LSTMNetParams(
    num_epochs=2000,
    learning_rate=0.01,
    dropout=0.3,
    input_size=29,
    hidden_size=20,
    hidden_layer=50,
    num_layers=1,
    num_classes=10,
    seq_length=X_train_t.shape[1],
    tensorboard=False
)
lstm = LSTMNet(lstm_params)
lstm_acc, lstm_f1, lstm_auc, lstm_mcc = model_assess(
    lstm, X_train_t, X_test_t, y_train_t, y_test_t, title="LSTM"
)
df_res.loc[len(df_res)] = ["LSTM", lstm_acc, lstm_f1, lstm_auc, lstm_mcc]


LSTM:
  Acc: 0.69
  F1: 0.69
  AUC score: 0.83
  Matth. corr. coeff.: 0.66


CNN Resnet

TODO

Save results

In [60]:
df_res.to_csv("data/classifier_results.csv")